In [2]:
import tensorflow as tf
keras = tf.keras
from keras.preprocessing.text import Tokenizer
from keras import layers, models

from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import pandas as pd

np.random.seed(999)

In [14]:
fake = pd.read_csv('data/Fake.csv')
true = pd.read_csv('data/True.csv')
fake["label"] = 1
true["label"] = 0 # Fake news is 1, true news is 0
true.head()

# Concatenate the dataframes
df = pd.concat([fake, true])
df["full text"] = df["title"] + df["text"]
df.head()

,title,text,subject,date,label,full text
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1,Pope Francis Just Called Out Donald Trump Dur...


In [17]:
i = np.random.rand(len(df)) < 0.8
train = df[i]
test = df[~i]
print("train data size: ", train.shape)
print("test data size: ", test.shape)

train data size:  (35695, 6)
test data size:  (9203, 6)


In [18]:
num_labels = 2
vocab_size = 25000
batch_size = 100


tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train["full text"])

x_train = tokenizer.texts_to_matrix(train["full text"], mode="tfidf")
x_test = tokenizer.texts_to_matrix(test["full text"], mode="tfidf")

encoder = LabelEncoder()
encoder.fit(train.label)
y_train = encoder.transform(train.label)
y_test = encoder.transform(test.label)

print("train shapes:", x_train.shape, y_train.shape)
print("test shapes:", x_test.shape, y_test.shape)
print("test first five labels:", y_test[:5])


train shapes: (35695, 25000) (35695,)
test shapes: (9203, 25000) (9203,)
test first five labels: [1 1 1 1 1]


In [20]:
model = models.Sequential()

model.add(layers.Dense(32, input_dim=vocab_size, kernel_initializer="normal", activation="relu"))
model.add(layers.Dense(1, kernel_initializer="normal", activation="sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
history = model.fit(x_train, y_train, 
                    batch_size=batch_size,
                    epochs=1,
                    verbose=1,
                    validation_split=0.1)

322/322 [==============================] - 12s 34ms/step - loss: 0.0898 - accuracy: 0.9725 - val_loss: 0.0637 - val_accuracy: 0.9826
